# Deep learning tweeter

In [39]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard



## Imports

In [1]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import copy
import pickle

warnings.filterwarnings('ignore')
seed = 5
np.random.seed(seed)

sys.path.append('..')
cufflinks.go_offline()

In [2]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, tokenize, procesar_corpus
from auxiliar import parameters
from sklearn.model_selection import KFold
from auxiliar.HtmlParser import HtmlParser

In [3]:
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard

In [4]:
import logging
import math
import pickle
import pandas as pd
import Levenshtein as lv
from nltk.stem import SnowballStemmer
from gensim.models import Doc2Vec
import gensim
from nltk.tokenize import word_tokenize
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from time import time, strftime

In [5]:
import nltk
# nltk.download()

## Config

In [6]:
polarity_dim = 2
# clasificadores=['lstm', '2lstm', '2dcnn', '2dcnn+lstm', 'cnn+lstm', 'bidirectionalLstm']
clasificadores=['lstm']
idf = True
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'deep_learning'

In [7]:
w2vec_file = 'data/w2vec.bin'
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
handler = logging.FileHandler('entrenamiento-%s.log' % strftime("%d-%m-%Y-%H-%M"), 'w', 'utf-8')  # or whatever
handler.setFormatter = logging.Formatter('%(name)s %(message)s')  # or whatever
root_logger.addHandler(handler)
stemmer = SnowballStemmer('spanish')

## Get data

Obtenemos los datos para realizar el entrenamiento y lo dividimos en dos subconjuntos: train y test

Dado que nuestro corpus de datos es muy pequeño, 4700 ejemplos para polaridades pos y neg, vamos a dividir el conjunto de datos de forma que utilicemos solo 100 ejemplos para el conjunto de test, asi no penalizaremos al conjunto de entrenamiento.



In [8]:
# cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
#     cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
#     cine = filter_corpus_small(cine.get_corpus())
# used_data = cine[:5000]

#Intentando obtener datos del archivo csv...
/home/suampa/Documentos/AnalysisCV/Corpus/../data/general-corpus.csv
#Datos recuperados!


In [9]:
data_corpus.groupby('polarity').agg({'index': 'count'}).iplot(kind='bar')

## Preprocess

Usamos la función axiliar prepro (VectorizerHelper.procesar_corpus) con los siguientes parámetros:

text, tagger, process_text, stop_words, negation, repeated_letters

In [48]:
def apply_prepro(data):
    return procesar_corpus(data, None, True, True, False, True)
data_corpus.content = data_corpus.content.apply(apply_prepro)

### Model initialization

In [100]:
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(w2vec_file, binary=True)

I0618 18:43:55.337981 139968762341184 utils_any2vec.py:341] loading projection weights from data/w2vec.bin
I0618 18:43:55.341667 139968762341184 smart_open_lib.py:293] {'uri': 'data/w2vec.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': {}}
I0618 18:44:14.323171 139968762341184 utils_any2vec.py:405] loaded (1000653, 300) matrix from data/w2vec.bin


### Tokenize texts

Utilizamos la funciona auxiliar tokenize que utiliza como parametros el conjunto de datos y un flag para activar el stemming.

In [229]:
def apply_tokenization(data):
    return tokenize(data, False)
tokens = data_corpus.content.apply(apply_tokenization)

In [230]:
token_df = pd.DataFrame([x for x in tokens]).transpose()
token_df.columns = pd.MultiIndex.from_arrays([data_corpus.polarity, token_df.columns])

In [231]:
print('palabras totales', token_df.count().sum())
print('media de palabras por texto', token_df.count().mean())

palabras totales 41522
media de palabras por texto 8.834468085106383


In [232]:
token_df.count().iplot(kind='histogram')

A pesar de que la mayoría de documentos tienen un máximo de 10 palabras, vamos a utilizar las 28 de máximo debido a la poca cantidad de datos de entrenamiento de la que disponemos

In [233]:
pd.DataFrame([token_df[x].count().sum() for x in token_df.columns.levels[0]]).iplot(kind='bar')

La cantidad de palabras por polaridad es bastante similar, aunque la diferencia puede llegar a suponer un problema en el resultado final según la distribución de los conjuntos de entrenamiento y de test

### w2vec process

Obtenemos las palabras que no se encuentran en el diccionario, probablemente por estar mal escritas como enorahuena en lugar de enhorabuena, en este caso los stems serían enorahuen y enhorabuen respectivamente, como vemos hay una diferencia de dos letras

Basandonos en esto vamos a establecer una diferencia máxima de 3 letras para considerar la palabra sustituible por la palabra correcta

In [238]:
# stem_vocab = np.array([stemmer.stem(x) for x in model.vocab])
stem_vocab = np.array([x for x in model.vocab])
stem_vocab_dict = dict.fromkeys(stem_vocab, 1)

In [239]:
asci_codes = [np.array([ord(x) for x in y]) for y in stem_vocab]

La palabra más larga del español según la RAE es electroencefalografista

In [240]:
len("electroencefalografista")

23

In [370]:
padded_asci_codes = tf.keras.preprocessing.sequence.pad_sequences(asci_codes, maxlen=23, padding='post')

In [69]:
from datetime import datetime
from tensorflow.python.keras.callbacks import TensorBoard
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index), 300, input_length=12, weights=[weight_matrix], name='Embedding', trainable=False),
    tf.keras.layers.LSTM(64, activation='relu', name='lstm'),
    tf.keras.layers.Dense(1, activation='sigmoid' ,name='dense')
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, embeddings_freq=1, write_images=True)

W0618 18:01:31.216591 139968762341184 callbacks.py:1475] `batch_size` is no longer needed in the `TensorBoard` Callback and will be ignored in TensorFlow 2.0.
W0618 18:01:31.219434 139968762341184 callbacks.py:1479] `embeddings_layer_names` is not supported in TensorFlow 2.0. Instead, all `Embedding` layers will be visualized.


#### No encontradas en vocabulario

In [242]:
not_in_vocab = pd.concat([token_df[d][token_df[d].apply(lambda x: x not in stem_vocab_dict)] for d in token_df.columns], axis=1)

In [243]:
print('total de palabras no encontradas en el vocabulario', not_in_vocab.count().sum())
print('media de palabras no encontradas en el conjunto', not_in_vocab.count().mean())

total de palabras no encontradas en el vocabulario 1935
media de palabras no encontradas en el conjunto 0.4117021276595745


In [244]:
pd.DataFrame([not_in_vocab[x].count().sum() for x in not_in_vocab.columns.levels[0]]).iplot(kind='bar')

In [245]:
not_in_vocab.columns = not_in_vocab.columns.droplevel()

In [246]:
not_in_vocab_words = pd.DataFrame(
    [x for sublist in [not_in_vocab[y].dropna().values for y in not_in_vocab.columns] for x in sublist]
).drop_duplicates()

#### Encontradas en vocabulario

In [474]:
in_vocab = pd.concat([token_df[d][token_df[d].apply(lambda x: x in stem_vocab_dict)] for d in token_df.columns], axis=1)

In [475]:
in_vocab.columns = in_vocab.columns.droplevel()

In [476]:
in_vocab_words = pd.DataFrame(
    [x for sublist in [in_vocab[y].dropna().values for y in in_vocab.columns] for x in sublist]
).drop_duplicates()

In [477]:
in_vocab.head()

,0,1,2,3,4,5,6,7,8,9,...,4690,4691,4692,4693,4694,4695,4696,4697,4698,4699
0,gracias,off,conozco,toca,buen,escaño,buenos,sistema,caca,buen,...,ya,rajoy,rick,NaN,nace,muy,más,crean,sorprendente,está
1,mar,pensando,adicto,grabación,día,listo,días,económico,ajuste,viernes,...,dos,da,santorum,será,jirafa,indignante,pobres,banco,huída,muy
2,NaN,regalito,drama,especial,primero,empezar,em,recorta,NaN,NaN,...,ganas,espalda,retira,presidente,primera,si,discriminar,productos,hoy,bien
3,NaN,sinde,ja,navideño,mandar,congreso,no,dinero,NaN,NaN,...,verte,post,campaña,NaN,su,repara,mujer,mujeres,senado,versión
4,NaN,va,ja,mari,abrazo,NaN,ira,prestaciones,NaN,NaN,...,rt,buzón,primarias,NaN,especie,hoy,NaN,cáncer,rajoy,gallega


#### Replace process

In [356]:
def replace_words(w):
    found_positions = np.where(stem_vocab == w)[0]
    return model[vocab_keys[found_positions[0]]]

In [364]:
def custom_levenshtein(word, dictionary):
    ascii_matrix = np.repeat([word], [dictionary.shape[0]], axis=0)
    difference = ascii_matrix - dictionary
    difference = np.where(difference != 0, 1, difference)
    difference_sum = np.sum(difference, axis=1)
    minval = np.min(difference_sum)
    minidx = np.argmin(difference_sum)
    if minval < 2 and minval > 0:
        return minidx
    else:
        return None

In [389]:
def find_coincidences(word, dictionary):
    min_len = max(len(word) - 1, 1)
    max_len = len(word) + 1
    dist = np.array([lv.distance(x, word) - x.startswith(word) if len(x) > min_len and len(x) < max_len else 999 for x in dictionary])
    minval = np.min(dist)
    if minval < 2 and minval > 0:
        minidx = np.argmin(dist)
#         root_logger.info('current word %s - found_distance %s - idx to replace %s word', word, minval, minidx)
        return dictionary[minidx], np.delete(dictionary, minidx)
    else:
        return None, dictionary
    
    
    

In [391]:
import gc
gc.collect()

58

In [392]:
dictionary = not_in_vocab_words.values.reshape(1,-1)[0]
ascii_dictionary = np.array([np.array([ord(x) for x in y]) for y in dictionary])
padded_dictionary = tf.keras.preprocessing.sequence.pad_sequences(ascii_dictionary, maxlen=23, padding="post")

found = dict()
with tqdm(total=len(padded_asci_codes)) as pbar:
    for i, val in enumerate(padded_asci_codes):
        pbar.update(1)
        coincidence = custom_levenshtein(val, padded_dictionary)
        if coincidence:
            found[dictionary[coincidence]] = model[stem_vocab[i]]
        if len(found) == dictionary.shape[0]:
            break

100%|██████████| 1000653/1000653 [12:42<00:00, 1312.29it/s]


In [480]:
gc.collect()

33990

In [381]:
dictionary = in_vocab_words.values.reshape(1,-1)[0]
w2vec_found = dict()
with tqdm(total=len(dictionary)) as pbar:
    for word in dictionary:
        pbar.update(1)
        w2vec_found[word] = model[word]

100%|██████████| 10833/10833 [00:00<00:00, 59712.08it/s]


In [478]:
in_vocab_replaced = in_vocab.applymap(lambda x: w2vec_found[x] if x in w2vec_found else math.nan)

In [481]:
not_in_vocab_replaced = not_in_vocab.applymap(lambda x: found[x] if x in found else math.nan)

In [483]:
in_vocab_replaced.update(not_in_vocab_replaced)

#### Replace nan positions

Todos los textos deben tener la misma longitud, por lo que aquellos textos que no tienen la longitud indicanda deben rellenarse con arrays de 0 de una dimension establecida (la misma para todas las palabras)

In [485]:
nan_pos = pd.DataFrame([in_vocab_replaced[c].isna() for c in in_vocab_replaced.columns]).transpose()

In [486]:
features = 300
zeros = np.zeros((features))
in_vocab_replaced.update(nan_pos.applymap(lambda x: zeros if x else math.nan))

In [581]:
in_vocab_replaced.to_pickle('tweeter_wemb.pkl')

## Split data

In [10]:
in_vocab_replaced = pd.read_pickle('tweeter_wemb.pkl')

In [12]:
in_vocab_replaced.head()

,0,1,2,3,4,5,6,7,8,9,...,4690,4691,4692,4693,4694,4695,4696,4697,4698,4699
0,"[0.123253495, 0.047755074, 0.18744704, -0.0576...","[-0.03825481, 0.4745884, 0.06159374, -0.211678...","[0.08140966, -0.2937571, 0.09323869, -0.070561...","[-0.08855907, -0.04540643, -0.03799705, 0.1179...","[-0.24166, -0.31998757, 0.05182405, -0.0511, 0...","[-0.30192834, -0.091402225, -0.07635854, -0.32...","[0.16187154, -0.15262279, 0.15911105, 0.136880...","[-0.03385875, -0.05679143, 0.15936868, 0.03850...","[0.08511154, -0.5224435, -0.1114207, -0.029714...","[-0.24166, -0.31998757, 0.05182405, -0.0511, 0...",...,"[0.08881656, -0.08638146, 0.19055837, 0.011414...","[0.24844696, -0.045677852, 0.023781389, -0.055...","[0.028837433, -0.1533759, -0.15925558, -0.0515...","[0.084870994, 0.018372163, -0.19153509, -0.140...","[0.19372217, -0.00981669, 0.0936164, 0.0327220...","[0.2661146, 0.10789581, 0.24465632, 0.09246798...","[0.0937914, -0.06750509, 0.11355269, -0.071939...","[0.05255723, -0.173229, -0.043076243, -0.07914...","[0.08545774, -0.18351299, 0.040896367, -0.2875...","[0.19491133, 0.13588089, 0.26361302, 0.0549132..."
1,"[-0.27860123, -0.0073691155, 0.07620924, -0.18...","[-0.13225149, 0.007982017, -0.15443377, -0.041...","[-0.22758521, 0.119482145, 0.07687994, -0.1562...","[0.19492386, 0.34440613, 0.05423296, 0.1994891...","[0.11887759, -0.062084418, 0.24743606, 0.08697...","[-0.03682754, 0.124864206, 0.08532753, 0.11072...","[0.29450724, -0.08953724, 0.22900815, -0.13844...","[-0.01992176, -0.38204813, 0.08824053, 0.02404...","[0.032929733, 0.071419924, -0.063104734, -0.08...","[0.16452287, 0.04510333, 0.17681116, -0.175676...",...,"[0.15753947, -0.11052575, 0.026150983, -0.0067...","[0.18700868, -0.04345352, -0.21399334, -0.0299...","[-0.115100406, 0.04057121, -0.051373735, -0.14...","[0.008335834, -0.13954785, 0.07759602, -0.0191...","[0.057889074, -0.0860811, 0.005500754, 0.19180...","[-0.09766652, -0.055921096, -0.18628502, -0.33...","[-0.14751814, 0.014919235, -0.06650046, -0.193...","[0.20185633, 0.16108106, 0.17906275, 0.0547961...","[-0.015799766, 0.09396541, 0.08841807, -0.0635...","[0.2661146, 0.10789581, 0.24465632, 0.09246798..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1472689, 0.035210133, -0.0905985, 0.235152...","[-0.19423386, -0.1605221, -0.122799665, -0.226...","[0.010192282, 0.011454537, -0.024294477, -0.14...","[-0.016709665, 0.04634203, 0.12317722, -0.1102...","[0.015565158, 0.013814226, 0.2070413, -0.10364...","[0.23736276, 0.31505284, -0.36881423, 0.341764...","[-0.2906534, 0.38485017, -0.32198793, -0.46555...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[-0.15636458, -0.14417209, 0.15087254, -0.0876...","[-0.18826456, -0.26298478, -0.30475214, -0.043...","[-0.32796225, 0.12644982, 0.1373805, -0.328892...","[-0.021904068, -0.28292415, 0.036686286, -0.12...","[0.31308195, 0.28671673, 0.061079856, -0.18517...","[0.060204167, -0.17087598, 0.2108287, -0.02707...","[-0.09252765, -0.06127216, -0.21862276, -0.010...","[0.10744205, 0.0075490335, -0.28650734, 0.2955...","[0.025537454, -0.1880749, 0.1568079, -0.178805...","[-0.06769281, -0.30116892, 0.08937262, 0.08235..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.038064547, -0.045823276, -0.07485215, -0.07...","[0.25501376, -0.011608973, -0.27930972, 0.3525...","[-0.18029697, 0.08496069, 0.01006801, -9.25252...","[-0.25586495, 0.28853804, 0.059111107, -0.0686...","[0.055961747, 0.03338554, 0.17965522, -0.18108...","[-0.0312227, -0.05622646, 0.2197303, -0.072537...","[-0.17901269, -0.015474343, 0.004863352, 0.029...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[-0.2848985, -0.20139082, -0.099536225, 0.1538...","[0.08226747, 0.031416256, 0.0009123865, -0.560...","[0.40826887, -0.085287675, 0.13206185, -0.1375...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.16144432, -0.056896593, -0.029683227, -0.1...","

Dado que solo tenemos 4700 elementos, lo que resulta ser un corpus bastante pequeño, vamos a quedarnos con solo 100 elementos como conjunto de test y utilizaremos el resto como conjunto de entrenamiento

In [83]:
split = in_vocab_replaced.shape[1] * 0.7

In [93]:
train_corpus = in_vocab_replaced.loc[:, :split - 1]
test_corpus = in_vocab_replaced.loc[:, split:]

In [107]:
print("corpus shape ", in_vocab_replaced.shape)
print("train_corpus shape ", train_corpus.shape)
print("test_corpus shape ", test_corpus.shape)

corpus shape  (28, 4700)
train_corpus shape  (28, 3290)
test_corpus shape  (28, 1410)


In [108]:
features=train_corpus[0][0].shape[0]
timesteps=train_corpus.shape[0]
elements=train_corpus.shape[1]
print("features ", features)
print("timesteps ", timesteps)
print("elements ", elements)

features  300
timesteps  28
elements  3290


In [109]:
wemb_x = np.array([np.concatenate(train_corpus[x].values) for x in train_corpus.columns])

In [110]:
wemb_test = np.array([np.concatenate(test_corpus[x].values) for x in test_corpus.columns])

In [111]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [112]:
tokenizer.fit_on_texts(data_corpus.content)

In [113]:
sequences = tokenizer.texts_to_sequences(data_corpus.content)

In [114]:
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=28, padding='post')

In [115]:
sequences.shape

(4700, 28)

In [116]:
pd.DataFrame(tokenizer.word_index.items(), columns=["word", "idx"]).to_pickle('word_index.pkl')

## Get final train data

In [117]:
wemb_x.shape

(3290, 8400)

In [118]:
x_train = wemb_x.reshape(elements, timesteps, features)

In [119]:
x_train.shape

(3290, 28, 300)

In [120]:
x_test = wemb_test.reshape(test_corpus.shape[1], timesteps, features)

In [121]:
x_test.shape

(1410, 28, 300)

In [122]:
from sklearn.preprocessing import LabelEncoder

In [123]:
def get_Y(corpus, polarity_dim=polarity_dim):
    if polarity_dim == 2:
        encoder = LabelEncoder()
        encoder.fit(corpus.polarity.values)
        Y = encoder.transform(corpus.polarity.values)
        return Y
    else:
        return to_categorical([x for x in train.polarity.values])

In [124]:
Y = get_Y(data_corpus, polarity_dim)

In [128]:
Y.shape

(4700,)

In [135]:
y_train = Y[:3290]
y_test = Y[3290:]

In [136]:
print('y_train shape ', y_train.shape)
print('y_test shape ', y_test.shape)

y_train shape  (3290,)
y_test shape  (1410,)


In [140]:
pd.DataFrame(y_train).iplot(kind='histogram')

In [141]:
pd.DataFrame(y_test).iplot(kind='histogram')

## Model baseline

Primero comprobamos como se comportaria la medida "loss" en un conjunto de validación. Para ello dividimos el conjunto de entrenamiento en un 90% entrenamiento 10% validación

In [137]:
colors=['red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue']

In [47]:
model_base = tf.keras.Sequential([
    tf.keras.layers.Dense(65, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(1, activation='sigmoid' ,name='dense')
])
model_base.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
logdir="logs/fit/base/base" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, embeddings_freq=1, write_images=True)
hist_base = model_base.fit(sequences[:-100], y_train, validation_split=0.2, shuffle=False, epochs=30,\
                           verbose=2, batch_size=128, callbacks=[tensorboard_callback])

NameError: name 'embedding_matrix' is not defined

In [524]:
model_base.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 300)           3644700   
_________________________________________________________________
flatten_2 (Flatten)          (None, 8400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 65)                546065    
_________________________________________________________________
dense (Dense)                (None, 1)                 66        
Total params: 4,190,831
Trainable params: 546,131
Non-trainable params: 3,644,700
_________________________________________________________________


### Definición de funciones

In [51]:
kf = KFold(n_splits=kfolds, shuffle=True, random_state=None) # realización de k-folds
folds = kf.split(x_train)

In [52]:
folds = [(x, y) for (x, y) in folds]

In [53]:
def convert_to_df(hist):
    cols = pd.MultiIndex.from_product([["step1", "step2", "step3", "step4", "step5", "step6", "step7", "step8", "step9", "step10"], hist[0].history.keys()])
    hist_df = pd.concat([pd.DataFrame(x.history) for x in  hist], axis=1)
    hist_df.columns = cols
    hist_df.head()
    return hist_df

In [66]:
def kfold_train_sequences(model_func, model_name, **params):
    evaluations = list()
    hists = list()
    i = 0
    for train_index, val_index in folds:
        i += 1
        model = model_func()
        train_x = sequences[train_index]
        train_y = y_train[train_index]
        val_x = sequences[val_index]
        val_y = y_train[val_index]
        
        logdir="logs/fit/"+ model_name+"/kfold" + str(i)
        print(logdir)
        tensorboard_callback = TensorBoard(log_dir=logdir, embeddings_freq=1, write_images=True)

        hist = model.fit(train_x, train_y, validation_data=(val_x, val_y), callbacks=[tensorboard_callback], **params)
        
        hists.append(hist)
        evaluations.append(model.evaluate(x_test, y_test))
    hist_df = convert_to_df(hists)
    hist_df.to_pickle('data/results/'+name+'/cine/' + base_dir + '/'+model_name+'_lstm.pkl')
    evas_df = pd.DataFrame(evaluations)
    evas_df.to_pickle('data/results/'+name+'/cine/' + base_dir + '/'+model_name+'_lstm_evas.pkl')
    return hist_df, evas_df

In [75]:
def kfold_train(model_func, model_name, **params):
    evaluations = list()
    hists = list()
    i = 0
    for train_index, val_index in folds:
        i += 1
        model = model_func()
        train_x = x_train[train_index]
        train_y = y_train[train_index]
        val_x = x_train[val_index]
        val_y = y_train[val_index]
        
        logdir="logs/fit/"+ model_name+"/kfold" + str(i)
        print(logdir)
        tensorboard_callback = TensorBoard(log_dir=logdir)

        hist = model.fit(train_x, train_y, validation_data=(val_x, val_y), callbacks=[tensorboard_callback], **params)
        
        hists.append(hist)
        evaluations.append(model.evaluate(x_test, y_test))
    hist_df = convert_to_df(hists)
    hist_df.to_pickle('data/results/'+name+'/cine/' + base_dir + '/'+model_name+'_lstm.pkl')
    evas_df = pd.DataFrame(evaluations)
    evas_df.to_pickle('data/results/'+name+'/cine/' + base_dir + '/'+model_name+'_lstm_evas.pkl')
    return hist_df, evas_df

In [56]:
def plot_loss(hist_df):
    hist_df.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
    print('mean ', hist_df.stack(level=0).mean())
    print('std ', hist_df.stack(level=0).std())

In [57]:
def plot_val(evas_df):
    evas_df[1].iplot()
    print('media ', evas_df[1].mean())
    print('std ', evas_df[1].std())

In [58]:
def process_results(hist, evas):
    plot_loss(hist)
    plot_val(evas)

### Definición de modelos

In [539]:
def create_lstm_val():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model
val_hist, val_evas = kfold_train(create_lstm_val, 'lstm_val', batch_size=128, epochs=30, shuffle=False, verbose=2)

logs/fit/lstm_val/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 5s - loss: 0.6875 - acc: 0.5727 - val_loss: 0.6763 - val_acc: 0.5739
Epoch 2/30
4140/4140 - 1s - loss: 0.7066 - acc: 0.6278 - val_loss: 0.6767 - val_acc: 0.6435
Epoch 3/30
4140/4140 - 1s - loss: 0.6711 - acc: 0.6304 - val_loss: 0.6087 - val_acc: 0.6978
Epoch 4/30
4140/4140 - 1s - loss: 0.6654 - acc: 0.7275 - val_loss: 0.6014 - val_acc: 0.7348
Epoch 5/30
4140/4140 - 1s - loss: 0.5932 - acc: 0.7408 - val_loss: 0.5504 - val_acc: 0.7652
Epoch 6/30
4140/4140 - 1s - loss: 0.5066 - acc: 0.7819 - val_loss: 0.4295 - val_acc: 0.7978
Epoch 7/30
4140/4140 - 1s - loss: 0.4089 - acc: 0.8092 - val_loss: 0.4255 - val_acc: 0.7870
Epoch 8/30
4140/4140 - 1s - loss: 0.3736 - acc: 0.8329 - val_loss: 0.4199 - val_acc: 0.8087
Epoch 9/30
4140/4140 - 1s - loss: 0.3552 - acc: 0.8386 - val_loss: 0.4245 - val_acc: 0.8043
Epoch 10/30
4140/4140 - 1s - loss: 0.3447 - acc: 0.8440 - val_loss: 0.4374 - val_acc: 0.8000
Epoch 1

Epoch 26/30
4140/4140 - 1s - loss: 0.2813 - acc: 0.8911 - val_loss: 0.3962 - val_acc: 0.8500
Epoch 27/30
4140/4140 - 1s - loss: 0.2741 - acc: 0.8973 - val_loss: 0.3975 - val_acc: 0.8543
Epoch 28/30
4140/4140 - 1s - loss: 0.2615 - acc: 0.9019 - val_loss: 0.4328 - val_acc: 0.8348
Epoch 29/30
4140/4140 - 1s - loss: 0.2471 - acc: 0.9104 - val_loss: 0.4341 - val_acc: 0.8348
Epoch 30/30
4140/4140 - 1s - loss: 0.2310 - acc: 0.9200 - val_loss: 0.4721 - val_acc: 0.8261
100/100 [==============================] - 0s 684us/sample - loss: 0.3213 - acc: 0.8500
logs/fit/lstm_val/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 24s - loss: 0.7066 - acc: 0.5746 - val_loss: 0.6834 - val_acc: 0.5978
Epoch 2/30
4140/4140 - 8s - loss: 0.6857 - acc: 0.5771 - val_loss: 0.6798 - val_acc: 0.6413
Epoch 3/30
4140/4140 - 7s - loss: 0.6869 - acc: 0.6118 - val_loss: 0.6780 - val_acc: 0.5978
Epoch 4/30
4140/4140 - 7s - loss: 0.6793 - acc: 0.5768 - val_loss: 0.6701 - val_acc: 0.5978
Epoch 

Epoch 20/30
4140/4140 - 7s - loss: 0.3009 - acc: 0.8964 - val_loss: 0.5251 - val_acc: 0.8239
Epoch 21/30
4140/4140 - 7s - loss: 0.3290 - acc: 0.8821 - val_loss: 0.4310 - val_acc: 0.8087
Epoch 22/30
4140/4140 - 7s - loss: 0.2813 - acc: 0.9114 - val_loss: 0.4637 - val_acc: 0.7783
Epoch 23/30
4140/4140 - 7s - loss: 0.2712 - acc: 0.9104 - val_loss: 0.4800 - val_acc: 0.7957
Epoch 24/30
4140/4140 - 8s - loss: 0.2576 - acc: 0.9147 - val_loss: 0.5295 - val_acc: 0.8152
Epoch 25/30
4140/4140 - 8s - loss: 0.2610 - acc: 0.9097 - val_loss: 0.4899 - val_acc: 0.8304
Epoch 26/30
4140/4140 - 7s - loss: 0.2462 - acc: 0.9123 - val_loss: 0.4705 - val_acc: 0.8087
Epoch 27/30
4140/4140 - 8s - loss: 0.2739 - acc: 0.8932 - val_loss: 0.4412 - val_acc: 0.8370
Epoch 28/30
4140/4140 - 7s - loss: 0.3151 - acc: 0.8829 - val_loss: 0.8931 - val_acc: 0.8326
Epoch 29/30
4140/4140 - 7s - loss: 0.2752 - acc: 0.9005 - val_loss: 0.5902 - val_acc: 0.8478
Epoch 30/30
4140/4140 - 7s - loss: 0.2238 - acc: 0.9217 - val_loss: 0.

Epoch 14/30
4140/4140 - 8s - loss: 0.5215 - acc: 0.8039 - val_loss: 0.5445 - val_acc: 0.7630
Epoch 15/30
4140/4140 - 7s - loss: 0.5058 - acc: 0.8106 - val_loss: 0.5343 - val_acc: 0.7696
Epoch 16/30
4140/4140 - 7s - loss: 0.4915 - acc: 0.8222 - val_loss: 0.5255 - val_acc: 0.7630
Epoch 17/30
4140/4140 - 7s - loss: 0.4770 - acc: 0.8278 - val_loss: 0.5126 - val_acc: 0.7761
Epoch 18/30
4140/4140 - 8s - loss: 0.4619 - acc: 0.8350 - val_loss: 0.4891 - val_acc: 0.8217
Epoch 19/30
4140/4140 - 8s - loss: 0.4522 - acc: 0.8386 - val_loss: 0.4791 - val_acc: 0.8283
Epoch 20/30
4140/4140 - 7s - loss: 0.4406 - acc: 0.8420 - val_loss: 0.4734 - val_acc: 0.8283
Epoch 21/30
4140/4140 - 7s - loss: 0.4323 - acc: 0.8447 - val_loss: 0.4718 - val_acc: 0.8065
Epoch 22/30
4140/4140 - 7s - loss: 0.4290 - acc: 0.8425 - val_loss: 0.4794 - val_acc: 0.7870
Epoch 23/30
4140/4140 - 7s - loss: 0.4481 - acc: 0.8256 - val_loss: 0.5326 - val_acc: 0.7261
Epoch 24/30
4140/4140 - 7s - loss: 0.3776 - acc: 0.8473 - val_loss: 0.

In [560]:
process_results(val_hist, val_evas)

mean  acc         0.817654
loss        0.397367
val_acc     0.782565
val_loss    0.569382
dtype: float64
std  acc         0.111895
loss        0.178906
val_acc     0.081457
val_loss    0.482332
dtype: float64


media  0.8219999969005585
std  0.0358391529207282


La reducción de la variable loss en el conjunto de entrenamiento produce un efecto de sobreentrenamiento que provoca un mayor error en el conjunto de validación

Algunas de las opciones para reducir este efecto son:
* Reducir la complejidad de la red neuronal
* Aplicar alguna clase de regularización al modelo
* Buscar una topología que se adapte mejor al problema
* Obtener más datos

#### Reducción complejidad modelo

In [78]:
def create_simpler_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
simpler_hist, simpler_evas = kfold_train(create_simpler_model, 'lstm_simple', batch_size=128, epochs=30, shuffle=False, verbose=2)

logs/fit/lstm_simple/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 10s - loss: 0.6913 - acc: 0.5773 - val_loss: 0.6691 - val_acc: 0.5957
Epoch 2/30
4140/4140 - 3s - loss: 0.6675 - acc: 0.5771 - val_loss: 0.6729 - val_acc: 0.5957
Epoch 3/30
4140/4140 - 4s - loss: 0.6523 - acc: 0.5771 - val_loss: 0.6207 - val_acc: 0.5957
Epoch 4/30
4140/4140 - 4s - loss: 0.6089 - acc: 0.5821 - val_loss: 0.5691 - val_acc: 0.6761
Epoch 5/30
4140/4140 - 7s - loss: 0.5496 - acc: 0.6601 - val_loss: 0.5520 - val_acc: 0.6500
Epoch 6/30
4140/4140 - 8s - loss: 0.5248 - acc: 0.7024 - val_loss: 0.5296 - val_acc: 0.7478
Epoch 7/30
4140/4140 - 8s - loss: 0.4890 - acc: 0.7862 - val_loss: 0.4961 - val_acc: 0.7913
Epoch 8/30
4140/4140 - 8s - loss: 0.4811 - acc: 0.8056 - val_loss: 0.4605 - val_acc: 0.7913
Epoch 9/30
4140/4140 - 8s - loss: 0.4498 - acc: 0.8116 - val_loss: 0.4297 - val_acc: 0.8000
Epoch 10/30
4140/4140 - 9s - loss: 0.4235 - acc: 0.8171 - val_loss: 0.4213 - val_acc: 0.8000
Epo

4140/4140 - 1s - loss: 0.4478 - acc: 0.8007 - val_loss: 0.5140 - val_acc: 0.8022
Epoch 26/30
4140/4140 - 1s - loss: 0.4356 - acc: 0.8147 - val_loss: 0.4595 - val_acc: 0.8174
Epoch 27/30
4140/4140 - 1s - loss: 0.4269 - acc: 0.8234 - val_loss: 0.4308 - val_acc: 0.8152
Epoch 28/30
4140/4140 - 1s - loss: 0.4162 - acc: 0.8256 - val_loss: 0.4356 - val_acc: 0.8130
Epoch 29/30
4140/4140 - 1s - loss: 0.4009 - acc: 0.8319 - val_loss: 0.4198 - val_acc: 0.8239
Epoch 30/30
4140/4140 - 1s - loss: 0.4015 - acc: 0.8350 - val_loss: 0.4215 - val_acc: 0.8109
100/100 [==============================] - 0s 651us/sample - loss: 0.4270 - acc: 0.8300
logs/fit/lstm_simple/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 37s - loss: 0.6919 - acc: 0.5800 - val_loss: 0.6919 - val_acc: 0.5326
Epoch 2/30
4140/4140 - 7s - loss: 0.6675 - acc: 0.5855 - val_loss: 0.6564 - val_acc: 0.5565
Epoch 3/30
4140/4140 - 7s - loss: 0.6178 - acc: 0.6621 - val_loss: 0.5967 - val_acc: 0.6413
Epoch 4/30
414

Epoch 19/30
4140/4140 - 7s - loss: 0.4794 - acc: 0.7957 - val_loss: 0.4810 - val_acc: 0.8261
Epoch 20/30
4140/4140 - 7s - loss: 0.4715 - acc: 0.8002 - val_loss: 0.4775 - val_acc: 0.8304
Epoch 21/30
4140/4140 - 7s - loss: 0.4644 - acc: 0.8039 - val_loss: 0.4756 - val_acc: 0.8326
Epoch 22/30
4140/4140 - 7s - loss: 0.4584 - acc: 0.8051 - val_loss: 0.4726 - val_acc: 0.8326
Epoch 23/30
4140/4140 - 7s - loss: 0.4529 - acc: 0.8116 - val_loss: 0.4685 - val_acc: 0.8304
Epoch 24/30
4140/4140 - 7s - loss: 0.4482 - acc: 0.8143 - val_loss: 0.4662 - val_acc: 0.8326
Epoch 25/30
4140/4140 - 7s - loss: 0.4427 - acc: 0.8169 - val_loss: 0.4649 - val_acc: 0.8348
Epoch 26/30
4140/4140 - 7s - loss: 0.4387 - acc: 0.8208 - val_loss: 0.4618 - val_acc: 0.8348
Epoch 27/30
4140/4140 - 7s - loss: 0.4340 - acc: 0.8237 - val_loss: 0.4589 - val_acc: 0.8370
Epoch 28/30
4140/4140 - 8s - loss: 0.4307 - acc: 0.8258 - val_loss: 0.4600 - val_acc: 0.8370
Epoch 29/30
4140/4140 - 7s - loss: 0.4259 - acc: 0.8290 - val_loss: 0.

Epoch 13/30
4140/4140 - 1s - loss: 0.3405 - acc: 0.8394 - val_loss: 0.3704 - val_acc: 0.8500
Epoch 14/30
4140/4140 - 1s - loss: 0.3301 - acc: 0.8452 - val_loss: 0.3767 - val_acc: 0.8457
Epoch 15/30
4140/4140 - 1s - loss: 0.3266 - acc: 0.8493 - val_loss: 0.3647 - val_acc: 0.8457
Epoch 16/30
4140/4140 - 1s - loss: 0.3223 - acc: 0.8510 - val_loss: 0.3712 - val_acc: 0.8500
Epoch 17/30
4140/4140 - 1s - loss: 0.3224 - acc: 0.8514 - val_loss: 0.3670 - val_acc: 0.8478
Epoch 18/30
4140/4140 - 1s - loss: 0.3066 - acc: 0.8548 - val_loss: 0.3752 - val_acc: 0.8587
Epoch 19/30
4140/4140 - 1s - loss: 0.3059 - acc: 0.8592 - val_loss: 0.3668 - val_acc: 0.8500
Epoch 20/30
4140/4140 - 1s - loss: 0.3066 - acc: 0.8560 - val_loss: 0.3619 - val_acc: 0.8500
Epoch 21/30
4140/4140 - 1s - loss: 0.2952 - acc: 0.8582 - val_loss: 0.3785 - val_acc: 0.8522
Epoch 22/30
4140/4140 - 1s - loss: 0.3035 - acc: 0.8638 - val_loss: 0.3693 - val_acc: 0.8478
Epoch 23/30
4140/4140 - 1s - loss: 0.3176 - acc: 0.8478 - val_loss: 0.

In [79]:
process_results(simpler_hist, simpler_evas)

mean  acc         0.787255
loss        0.436262
val_acc     0.774638
val_loss    0.472216
dtype: float64
std  acc         0.092440
loss        0.131182
val_acc     0.084163
val_loss    0.098407
dtype: float64


media  0.8080000042915344
std  0.0456557164254997


LA reducción de dimensionalidad de la red neuronal permite reducir la velocidad de aprendizaje sobre el conjunto de entrenamiento reduciendo así la perdida en el conjunto de validación.

Si lo compramos con el resultado anterior vemos que en general se reducen los picos negativos, en un 0.30 como minimo, sin embargo la convergencia sigue estando sobre 0.4 en los mejores casos

#### Regularización por dropout

In [542]:
def create_drop_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model
drop_hist, drop_evas = kfold_train(create_drop_model, 'dropout_lstm', batch_size=128, epochs=30, shuffle=False, verbose=1)

W0619 08:13:41.216991 139968762341184 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


logs/fit/dropout_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 27s 6ms/sample - loss: 0.6919 - acc: 0.5790 - val_loss: 0.6903 - val_acc: 0.5739
Epoch 2/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6849 - acc: 0.5763 - val_loss: 0.6808 - val_acc: 0.5739
Epoch 3/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6635 - acc: 0.5787 - val_loss: 0.6189 - val_acc: 0.5739
Epoch 4/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6287 - acc: 0.5795 - val_loss: 0.5961 - val_acc: 0.5739
Epoch 5/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6070 - acc: 0.5783 - val_loss: 0.5408 - val_acc: 0.5739
Epoch 6/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.5725 - acc: 0.5790 - val_loss: 0.5254 - val_acc: 0.5739
Epoch 7/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6633 - acc: 0.5857 - val_lo

W0619 08:17:33.814997 139968762341184 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


logs/fit/dropout_lstm/kfold2
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 13s 3ms/sample - loss: 0.6922 - acc: 0.5746 - val_loss: 0.6913 - val_acc: 0.5696
Epoch 2/30
4140/4140 [==============================] - 4s 1ms/sample - loss: 0.6902 - acc: 0.5800 - val_loss: 0.6896 - val_acc: 0.5696
Epoch 3/30
4140/4140 [==============================] - 6s 1ms/sample - loss: 0.6881 - acc: 0.5800 - val_loss: 0.6875 - val_acc: 0.5696
Epoch 4/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6833 - acc: 0.5800 - val_loss: 0.6817 - val_acc: 0.5870
Epoch 5/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6557 - acc: 0.6529 - val_loss: 0.6146 - val_acc: 0.7348
Epoch 6/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6117 - acc: 0.6766 - val_loss: 0.5228 - val_acc: 0.7717
Epoch 7/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.5589 - acc: 0.7056 - val_lo

W0619 08:21:36.685403 139968762341184 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


...

logs/fit/dropout_lstm/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6921 - acc: 0.5744 - val_loss: 0.6900 - val_acc: 0.5978
Epoch 2/30
4140/4140 [==============================] - 1s 302us/sample - loss: 0.6889 - acc: 0.5877 - val_loss: 0.6823 - val_acc: 0.6587
Epoch 3/30
4140/4140 [==============================] - 1s 301us/sample - loss: 0.6812 - acc: 0.6353 - val_loss: 0.6818 - val_acc: 0.6130
Epoch 4/30
4140/4140 [==============================] - 1s 301us/sample - loss: 0.6844 - acc: 0.5809 - val_loss: 0.6780 - val_acc: 0.6109
Epoch 5/30
4140/4140 [==============================] - 1s 330us/sample - loss: 0.6752 - acc: 0.6188 - val_loss: 0.6594 - val_acc: 0.7239
Epoch 6/30
4140/4140 [==============================] - 1s 329us/sample - loss: 0.6541 - acc: 0.6720 - val_loss: 0.6039 - val_acc: 0.7870
Epoch 7/30
4140/4140 [==============================] - 2s 473us/sample - loss: 0.6371 - acc: 0.68

W0619 08:28:50.232286 139968762341184 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


logs/fit/dropout_lstm/kfold5
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6921 - acc: 0.5773 - val_loss: 0.6913 - val_acc: 0.5609
Epoch 2/30
4140/4140 [==============================] - 1s 304us/sample - loss: 0.6897 - acc: 0.5809 - val_loss: 0.6885 - val_acc: 0.5609
Epoch 3/30
4140/4140 [==============================] - 1s 302us/sample - loss: 0.6807 - acc: 0.6278 - val_loss: 0.6618 - val_acc: 0.6935
Epoch 4/30
4140/4140 [==============================] - 1s 302us/sample - loss: 0.6613 - acc: 0.6882 - val_loss: 0.6256 - val_acc: 0.7217
Epoch 5/30
4140/4140 [==============================] - 1s 301us/sample - loss: 0.6315 - acc: 0.6973 - val_loss: 0.5938 - val_acc: 0.7522
Epoch 6/30
4140/4140 [==============================] - 1s 300us/sample - loss: 0.6001 - acc: 0.6978 - val_loss: 0.5125 - val_acc: 0.7804
Epoch 7/30
4140/4140 [==============================] - 1s 308us/sample - loss: 0.5605 - acc: 0.72

Epoch 29/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.4240 - acc: 0.8089 - val_loss: 0.4985 - val_acc: 0.8283
Epoch 30/30
100/100 [==============================] - 1s 5ms/sample - loss: 0.6181 - acc: 0.8100
logs/fit/dropout_lstm/kfold7
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 35s 8ms/sample - loss: 0.6921 - acc: 0.5758 - val_loss: 0.6911 - val_acc: 0.5717
Epoch 2/30
4140/4140 [==============================] - 1s 305us/sample - loss: 0.6892 - acc: 0.5829 - val_loss: 0.6771 - val_acc: 0.6217
Epoch 3/30
4140/4140 [==============================] - 1s 298us/sample - loss: 1.0435 - acc: 0.7012 - val_loss: 0.6174 - val_acc: 0.7413
Epoch 4/30
4140/4140 [==============================] - 1s 303us/sample - loss: 0.6487 - acc: 0.6990 - val_loss: 0.6412 - val_acc: 0.7152
Epoch 5/30
4140/4140 [==============================] - 1s 304us/sample - loss: 0.6327 - acc: 0.7072 - val_loss: 0.6322 - val_acc: 0.7196


4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3732 - acc: 0.8063 - val_loss: 0.3844 - val_acc: 0.8217
Epoch 27/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3692 - acc: 0.8147 - val_loss: 0.3951 - val_acc: 0.8217
Epoch 28/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3493 - acc: 0.8234 - val_loss: 0.3944 - val_acc: 0.8370
Epoch 29/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3508 - acc: 0.8227 - val_loss: 0.4037 - val_acc: 0.8217
Epoch 30/30
100/100 [==============================] - 1s 6ms/sample - loss: 0.3673 - acc: 0.8500
logs/fit/dropout_lstm/kfold9
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 53s 13ms/sample - loss: 0.6921 - acc: 0.5778 - val_loss: 0.6915 - val_acc: 0.5587
Epoch 2/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6894 - acc: 0.5812 - val_loss: 0.6893 - val_acc: 0.5587
Epoch 3/30
4140/

In [555]:
process_results(drop_hist, drop_evas)

mean  acc         0.731907
loss        0.514644
val_acc     0.756217
val_loss    0.498160
dtype: float64
std  acc         0.083188
loss        0.108404
val_acc     0.091526
val_loss    0.103988
dtype: float64


media  0.8009999990463257
std  0.10826408435368169


Un valor tan alto de dropout (0.7) provoca peores resultados

In [543]:
def create_drop_model_2():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model
drop2_hist, drop2_evas = kfold_train(create_drop_model_2, 'dropout2_lstm', batch_size=128, epochs=30, shuffle=False, verbose=1)


logs/fit/dropout2_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 55s 13ms/sample - loss: 0.6921 - acc: 0.5727 - val_loss: 0.6906 - val_acc: 0.5739
Epoch 2/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6845 - acc: 0.6329 - val_loss: 0.6676 - val_acc: 0.6957
Epoch 3/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6607 - acc: 0.6778 - val_loss: 0.6157 - val_acc: 0.7196
Epoch 4/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6282 - acc: 0.7012 - val_loss: 0.5673 - val_acc: 0.7413
Epoch 5/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6007 - acc: 0.7343 - val_loss: 0.5576 - val_acc: 0.7370
Epoch 6/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.5136 - acc: 0.7512 - val_loss: 0.4621 - val_acc: 0.7717
Epoch 7/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.4643 - acc: 0.7729 - val_

4140/4140 [==============================] - 8s 2ms/sample - loss: 0.4177 - acc: 0.7826 - val_loss: 0.4174 - val_acc: 0.7870
Epoch 30/30
100/100 [==============================] - 0s 4ms/sample - loss: 0.3634 - acc: 0.8100
logs/fit/dropout2_lstm/kfold3
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 10s 2ms/sample - loss: 0.6920 - acc: 0.5754 - val_loss: 0.6892 - val_acc: 0.6087
Epoch 2/30
4140/4140 [==============================] - 1s 306us/sample - loss: 0.6788 - acc: 0.5756 - val_loss: 0.6466 - val_acc: 0.6087
Epoch 3/30
4140/4140 [==============================] - 1s 336us/sample - loss: 0.6328 - acc: 0.5754 - val_loss: 0.5975 - val_acc: 0.6087
Epoch 4/30
4140/4140 [==============================] - 1s 337us/sample - loss: 0.6136 - acc: 0.5754 - val_loss: 0.5744 - val_acc: 0.6087
Epoch 5/30
4140/4140 [==============================] - 2s 447us/sample - loss: 0.5673 - acc: 0.5749 - val_loss: 0.5212 - val_acc: 0.6087
Epoch 6/30


4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2770 - acc: 0.8862 - val_loss: 0.4954 - val_acc: 0.8109
Epoch 24/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.2755 - acc: 0.8915 - val_loss: 0.4978 - val_acc: 0.8087
Epoch 25/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2733 - acc: 0.8889 - val_loss: 0.5030 - val_acc: 0.8109
Epoch 26/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2538 - acc: 0.8973 - val_loss: 0.5613 - val_acc: 0.8065
Epoch 27/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2458 - acc: 0.9039 - val_loss: 0.6001 - val_acc: 0.8130
Epoch 28/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2559 - acc: 0.9002 - val_loss: 0.5447 - val_acc: 0.8065
Epoch 29/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.2606 - acc: 0.8976 - val_loss: 0.5027 - val_acc: 0.8022
Epoch 30/30
100/100 [==============================] 

4140/4140 [==============================] - 2s 362us/sample - loss: 0.5578 - acc: 0.7486 - val_loss: 0.5572 - val_acc: 0.7500
Epoch 18/30
4140/4140 [==============================] - 1s 337us/sample - loss: 0.5359 - acc: 0.7722 - val_loss: 0.5268 - val_acc: 0.7630
Epoch 19/30
4140/4140 [==============================] - 2s 413us/sample - loss: 0.5174 - acc: 0.7826 - val_loss: 0.4982 - val_acc: 0.7957
Epoch 20/30
4140/4140 [==============================] - 2s 484us/sample - loss: 0.4894 - acc: 0.8104 - val_loss: 0.4644 - val_acc: 0.8065
Epoch 21/30
4140/4140 [==============================] - 4s 997us/sample - loss: 0.4793 - acc: 0.8150 - val_loss: 0.4961 - val_acc: 0.7783
Epoch 22/30
4140/4140 [==============================] - 4s 1ms/sample - loss: 0.4644 - acc: 0.8239 - val_loss: 0.4494 - val_acc: 0.8043
Epoch 23/30
4140/4140 [==============================] - 6s 1ms/sample - loss: 0.4682 - acc: 0.8239 - val_loss: 0.4962 - val_acc: 0.8109
Epoch 24/30
4140/4140 [====================

In [554]:
process_results(drop2_hist, drop2_evas)

mean  acc         0.777572
loss        0.452812
val_acc     0.765080
val_loss    0.507004
dtype: float64
std  acc         0.102955
loss        0.137817
val_acc     0.081349
val_loss    0.096892
dtype: float64


media  0.8259999930858613
std  0.026749875386270022


Cuando reducimos el valor de dropout a 0.2 vemos que el val_loss es menos estable, y presenta picos más altos que el dropout mayor

#### Batch Normalization

In [544]:
def create_bn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model
bn_hist, bn_evas = kfold_train(create_bn_model, 'bn_lstm', batch_size=128, epochs=30, shuffle=False, verbose=1)

logs/fit/bn_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 62s 15ms/sample - loss: 0.6913 - acc: 0.5442 - val_loss: 0.6905 - val_acc: 0.5739
Epoch 2/30
4140/4140 [==============================] - 5s 1ms/sample - loss: 0.6814 - acc: 0.6116 - val_loss: 0.6855 - val_acc: 0.5783
Epoch 3/30
4140/4140 [==============================] - 1s 311us/sample - loss: 0.6528 - acc: 0.6614 - val_loss: 0.6640 - val_acc: 0.7022
Epoch 4/30
4140/4140 [==============================] - 1s 306us/sample - loss: 0.6072 - acc: 0.7046 - val_loss: 0.6153 - val_acc: 0.7543
Epoch 5/30
4140/4140 [==============================] - 1s 312us/sample - loss: 0.5548 - acc: 0.7534 - val_loss: 0.5415 - val_acc: 0.8087
Epoch 6/30
4140/4140 [==============================] - 1s 316us/sample - loss: 0.5103 - acc: 0.7722 - val_loss: 0.5039 - val_acc: 0.8022
Epoch 7/30
4140/4140 [==============================] - 1s 307us/sample - loss: 0.4814 - acc: 0.7879 - 

Epoch 29/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.3185 - acc: 0.8906 - val_loss: 0.4243 - val_acc: 0.8435
Epoch 30/30
100/100 [==============================] - 1s 6ms/sample - loss: 0.4094 - acc: 0.8300
logs/fit/bn_lstm/kfold3
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 61s 15ms/sample - loss: 0.6915 - acc: 0.5623 - val_loss: 0.6886 - val_acc: 0.6087
Epoch 2/30
4140/4140 [==============================] - 1s 318us/sample - loss: 0.6757 - acc: 0.6254 - val_loss: 0.6788 - val_acc: 0.6196
Epoch 3/30
4140/4140 [==============================] - 1s 309us/sample - loss: 0.6214 - acc: 0.7036 - val_loss: 0.6570 - val_acc: 0.6217
Epoch 4/30
4140/4140 [==============================] - 1s 314us/sample - loss: 0.5691 - acc: 0.7374 - val_loss: 0.6198 - val_acc: 0.6870
Epoch 5/30
4140/4140 [==============================] - 1s 310us/sample - loss: 0.5292 - acc: 0.7527 - val_loss: 0.5663 - val_acc: 0.7587
Epoc

4140/4140 [==============================] - 9s 2ms/sample - loss: 0.3782 - acc: 0.8399 - val_loss: 0.4271 - val_acc: 0.8283
Epoch 27/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.3775 - acc: 0.8514 - val_loss: 0.4495 - val_acc: 0.8283
Epoch 28/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3802 - acc: 0.8483 - val_loss: 0.4182 - val_acc: 0.8196
Epoch 29/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.3604 - acc: 0.8589 - val_loss: 0.4154 - val_acc: 0.8152
Epoch 30/30
100/100 [==============================] - 0s 5ms/sample - loss: 0.3709 - acc: 0.8600
logs/fit/bn_lstm/kfold5
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 [==============================] - 49s 12ms/sample - loss: 0.6874 - acc: 0.5722 - val_loss: 0.6896 - val_acc: 0.5609
Epoch 2/30
4140/4140 [==============================] - 1s 311us/sample - loss: 0.6540 - acc: 0.6746 - val_loss: 0.6474 - val_acc: 0.7435
Epoch 3/30
4140/414

4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6465 - acc: 0.6302 - val_loss: 0.6274 - val_acc: 0.6522
Epoch 21/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6321 - acc: 0.6522 - val_loss: 0.6236 - val_acc: 0.6652
Epoch 22/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6380 - acc: 0.6459 - val_loss: 0.6187 - val_acc: 0.6891
Epoch 23/30
4140/4140 [==============================] - 8s 2ms/sample - loss: 0.6258 - acc: 0.6524 - val_loss: 0.6125 - val_acc: 0.6978
Epoch 24/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6275 - acc: 0.6512 - val_loss: 0.6074 - val_acc: 0.6978
Epoch 25/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6231 - acc: 0.6505 - val_loss: 0.6120 - val_acc: 0.6870
Epoch 26/30
4140/4140 [==============================] - 9s 2ms/sample - loss: 0.6296 - acc: 0.6457 - val_loss: 0.6077 - val_acc: 0.6957
Epoch 27/30
4140/4140 [==============================

In [553]:
bn_evas

,0,1
0,0.449343,0.82
1,0.409371,0.83
2,0.431744,0.85
3,0.370904,0.86
4,0.503804,0.81
5,0.340413,0.87
6,0.561920,0.80
7,0.547365,0.79
8,0.379911,0.84
9,0.442909,0.81


In [552]:
process_results(bn_hist, bn_evas)

mean  acc         0.788700
loss        0.455791
val_acc     0.787696
val_loss    0.478266
dtype: float64
std  acc         0.081273
loss        0.112855
val_acc     0.073134
val_loss    0.093776
dtype: float64


media  0.8280000030994416
std  0.02658320128150446


#### Weights initialization

Probamos la inicialización de pesos por el algoritmo Xavier (glorot_normal)

In [559]:
def glorot_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
gl_hist, gl_evas = kfold_train(glorot_model, 'glorot_lstm', batch_size=128, epochs=30, shuffle=False, verbose=2)

logs/fit/glorot_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 62s - loss: 0.6781 - acc: 0.6056 - val_loss: 0.6842 - val_acc: 0.5761
Epoch 2/30
4140/4140 - 7s - loss: 0.6554 - acc: 0.6353 - val_loss: 0.6738 - val_acc: 0.5848
Epoch 3/30
4140/4140 - 7s - loss: 0.6219 - acc: 0.6749 - val_loss: 0.6635 - val_acc: 0.6109
Epoch 4/30
4140/4140 - 8s - loss: 0.5743 - acc: 0.7191 - val_loss: 0.6545 - val_acc: 0.6522
Epoch 5/30
4140/4140 - 8s - loss: 0.5452 - acc: 0.7449 - val_loss: 0.6271 - val_acc: 0.7391
Epoch 6/30
4140/4140 - 7s - loss: 0.5162 - acc: 0.7599 - val_loss: 0.6074 - val_acc: 0.7565
Epoch 7/30
4140/4140 - 7s - loss: 0.5007 - acc: 0.7783 - val_loss: 0.5744 - val_acc: 0.8022
Epoch 8/30
4140/4140 - 7s - loss: 0.4704 - acc: 0.7942 - val_loss: 0.5473 - val_acc: 0.8217
Epoch 9/30
4140/4140 - 7s - loss: 0.4627 - acc: 0.8080 - val_loss: 0.5279 - val_acc: 0.8239
Epoch 10/30
4140/4140 - 6s - loss: 0.4443 - acc: 0.8094 - val_loss: 0.4928 - val_acc: 0.8196
Epo

4140/4140 - 8s - loss: 0.3817 - acc: 0.8273 - val_loss: 0.4575 - val_acc: 0.7978
Epoch 26/30
4140/4140 - 8s - loss: 0.3712 - acc: 0.8382 - val_loss: 0.4851 - val_acc: 0.7978
Epoch 27/30
4140/4140 - 8s - loss: 0.3700 - acc: 0.8432 - val_loss: 0.4486 - val_acc: 0.7935
Epoch 28/30
4140/4140 - 8s - loss: 0.3647 - acc: 0.8382 - val_loss: 0.4583 - val_acc: 0.8043
Epoch 29/30
4140/4140 - 6s - loss: 0.3512 - acc: 0.8466 - val_loss: 0.4573 - val_acc: 0.8087
Epoch 30/30
4140/4140 - 1s - loss: 0.3351 - acc: 0.8560 - val_loss: 0.4379 - val_acc: 0.8326
100/100 [==============================] - 0s 661us/sample - loss: 0.4745 - acc: 0.8200
logs/fit/glorot_lstm/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 33s - loss: 0.6887 - acc: 0.5638 - val_loss: 0.6846 - val_acc: 0.5978
Epoch 2/30
4140/4140 - 8s - loss: 0.6699 - acc: 0.6179 - val_loss: 0.6761 - val_acc: 0.5978
Epoch 3/30
4140/4140 - 8s - loss: 0.6376 - acc: 0.6519 - val_loss: 0.6596 - val_acc: 0.6348
Epoch 4/30
414

Epoch 19/30
4140/4140 - 1s - loss: 0.3802 - acc: 0.8406 - val_loss: 0.4395 - val_acc: 0.7826
Epoch 20/30
4140/4140 - 1s - loss: 0.3602 - acc: 0.8517 - val_loss: 0.3983 - val_acc: 0.8326
Epoch 21/30
4140/4140 - 1s - loss: 0.3471 - acc: 0.8601 - val_loss: 0.4221 - val_acc: 0.8478
Epoch 22/30
4140/4140 - 1s - loss: 0.3629 - acc: 0.8488 - val_loss: 0.3960 - val_acc: 0.8326
Epoch 23/30
4140/4140 - 1s - loss: 0.3434 - acc: 0.8606 - val_loss: 0.3880 - val_acc: 0.8565
Epoch 24/30
4140/4140 - 1s - loss: 0.3438 - acc: 0.8592 - val_loss: 0.3963 - val_acc: 0.8478
Epoch 25/30
4140/4140 - 1s - loss: 0.3419 - acc: 0.8645 - val_loss: 0.3981 - val_acc: 0.8565
Epoch 26/30
4140/4140 - 1s - loss: 0.3578 - acc: 0.8546 - val_loss: 0.4740 - val_acc: 0.7652
Epoch 27/30
4140/4140 - 1s - loss: 0.3360 - acc: 0.8664 - val_loss: 0.4444 - val_acc: 0.8174
Epoch 28/30
4140/4140 - 1s - loss: 0.3291 - acc: 0.8676 - val_loss: 0.4342 - val_acc: 0.8152
Epoch 29/30
4140/4140 - 1s - loss: 0.3166 - acc: 0.8732 - val_loss: 0.

4140/4140 - 1s - loss: 0.4829 - acc: 0.7841 - val_loss: 0.4670 - val_acc: 0.8174
Epoch 13/30
4140/4140 - 1s - loss: 0.4559 - acc: 0.7978 - val_loss: 0.4508 - val_acc: 0.8196
Epoch 14/30
4140/4140 - 1s - loss: 0.4500 - acc: 0.8060 - val_loss: 0.4327 - val_acc: 0.8065
Epoch 15/30
4140/4140 - 1s - loss: 0.4344 - acc: 0.8130 - val_loss: 0.4234 - val_acc: 0.8087
Epoch 16/30
4140/4140 - 1s - loss: 0.4207 - acc: 0.8234 - val_loss: 0.4085 - val_acc: 0.8283
Epoch 17/30
4140/4140 - 1s - loss: 0.4260 - acc: 0.8159 - val_loss: 0.4442 - val_acc: 0.8043
Epoch 18/30
4140/4140 - 1s - loss: 0.4116 - acc: 0.8213 - val_loss: 0.4264 - val_acc: 0.8109
Epoch 19/30
4140/4140 - 1s - loss: 0.3934 - acc: 0.8391 - val_loss: 0.4340 - val_acc: 0.8130
Epoch 20/30
4140/4140 - 1s - loss: 0.3889 - acc: 0.8357 - val_loss: 0.4242 - val_acc: 0.8217
Epoch 21/30
4140/4140 - 1s - loss: 0.3702 - acc: 0.8502 - val_loss: 0.4151 - val_acc: 0.8217
Epoch 22/30
4140/4140 - 1s - loss: 0.3762 - acc: 0.8440 - val_loss: 0.4538 - val_a

In [564]:
process_results(gl_hist, gl_evas)

mean  acc         0.791872
loss        0.446528
val_acc     0.781109
val_loss    0.490319
dtype: float64
std  acc         0.072161
loss        0.099221
val_acc     0.077469
val_loss    0.090155
dtype: float64


media  0.8010000050067901
std  0.05425249575564484


In [61]:
def glorot_model_wo_bn():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
gl__wobn_hist, gl_wobn_evas = kfold_train(glorot_model_wo_bn, 'glorot__wobn_lstm', batch_size=128, epochs=30, shuffle=False, verbose=2)

logs/fit/glorot__wobn_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 16s - loss: 0.6911 - acc: 0.5746 - val_loss: 0.6864 - val_acc: 0.5957
Epoch 2/30
4140/4140 - 7s - loss: 0.6850 - acc: 0.5768 - val_loss: 0.6736 - val_acc: 0.5957
Epoch 3/30
4140/4140 - 7s - loss: 0.6670 - acc: 0.5906 - val_loss: 0.6234 - val_acc: 0.6739
Epoch 4/30
4140/4140 - 7s - loss: 0.6324 - acc: 0.6543 - val_loss: 0.5719 - val_acc: 0.7348
Epoch 5/30
4140/4140 - 5s - loss: 0.6061 - acc: 0.6964 - val_loss: 0.5504 - val_acc: 0.7674
Epoch 6/30
4140/4140 - 1s - loss: 0.5819 - acc: 0.6923 - val_loss: 0.4730 - val_acc: 0.7826
Epoch 7/30
4140/4140 - 1s - loss: 0.5457 - acc: 0.7155 - val_loss: 0.4479 - val_acc: 0.7848
Epoch 8/30
4140/4140 - 1s - loss: 0.5192 - acc: 0.7469 - val_loss: 0.4264 - val_acc: 0.8065
Epoch 9/30
4140/4140 - 1s - loss: 0.5023 - acc: 0.7464 - val_loss: 0.4133 - val_acc: 0.8087
Epoch 10/30
4140/4140 - 1s - loss: 0.4856 - acc: 0.7640 - val_loss: 0.4002 - val_acc: 0.81

Epoch 25/30
4140/4140 - 7s - loss: 0.4602 - acc: 0.7795 - val_loss: 0.3882 - val_acc: 0.8326
Epoch 26/30
4140/4140 - 7s - loss: 0.4658 - acc: 0.7819 - val_loss: 0.3996 - val_acc: 0.8152
Epoch 27/30
4140/4140 - 7s - loss: 0.4439 - acc: 0.7930 - val_loss: 0.3968 - val_acc: 0.8152
Epoch 28/30
4140/4140 - 7s - loss: 0.4496 - acc: 0.7884 - val_loss: 0.3849 - val_acc: 0.8283
Epoch 29/30
4140/4140 - 7s - loss: 0.4380 - acc: 0.7903 - val_loss: 0.3698 - val_acc: 0.8348
Epoch 30/30
4140/4140 - 7s - loss: 0.4455 - acc: 0.7853 - val_loss: 0.3731 - val_acc: 0.8391
100/100 [==============================] - 0s 5ms/sample - loss: 0.3614 - acc: 0.8400
logs/fit/glorot__wobn_lstm/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 18s - loss: 0.6910 - acc: 0.5800 - val_loss: 0.6914 - val_acc: 0.5326
Epoch 2/30
4140/4140 - 7s - loss: 0.6875 - acc: 0.5814 - val_loss: 0.6861 - val_acc: 0.5370
Epoch 3/30
4140/4140 - 7s - loss: 0.6895 - acc: 0.5937 - val_loss: 0.6723 - val_acc: 0.534

Epoch 18/30
4140/4140 - 1s - loss: 0.4290 - acc: 0.7742 - val_loss: 0.4428 - val_acc: 0.8261
Epoch 19/30
4140/4140 - 1s - loss: 0.4189 - acc: 0.7693 - val_loss: 0.4795 - val_acc: 0.8217
Epoch 20/30
4140/4140 - 1s - loss: 0.4177 - acc: 0.7696 - val_loss: 0.4633 - val_acc: 0.8283
Epoch 21/30
4140/4140 - 1s - loss: 0.4175 - acc: 0.7650 - val_loss: 0.4052 - val_acc: 0.8152
Epoch 22/30
4140/4140 - 1s - loss: 0.4015 - acc: 0.7800 - val_loss: 0.4331 - val_acc: 0.8239
Epoch 23/30
4140/4140 - 1s - loss: 0.3983 - acc: 0.7802 - val_loss: 0.4276 - val_acc: 0.8348
Epoch 24/30
4140/4140 - 1s - loss: 0.3868 - acc: 0.7923 - val_loss: 0.5409 - val_acc: 0.8261
Epoch 25/30
4140/4140 - 1s - loss: 0.4035 - acc: 0.7816 - val_loss: 0.4822 - val_acc: 0.8304
Epoch 26/30
4140/4140 - 1s - loss: 0.3822 - acc: 0.7983 - val_loss: 0.5901 - val_acc: 0.8283
Epoch 27/30
4140/4140 - 1s - loss: 0.3845 - acc: 0.8048 - val_loss: 0.5029 - val_acc: 0.8239
Epoch 28/30
4140/4140 - 1s - loss: 0.3851 - acc: 0.8138 - val_loss: 0.

Epoch 11/30
4140/4140 - 7s - loss: 0.5809 - acc: 0.6217 - val_loss: 0.5571 - val_acc: 0.7065
Epoch 12/30
4140/4140 - 7s - loss: 0.5721 - acc: 0.6377 - val_loss: 0.5471 - val_acc: 0.7500
Epoch 13/30
4140/4140 - 7s - loss: 0.5762 - acc: 0.6587 - val_loss: 0.5390 - val_acc: 0.7783
Epoch 14/30
4140/4140 - 7s - loss: 0.5619 - acc: 0.6758 - val_loss: 0.5282 - val_acc: 0.7891
Epoch 15/30
4140/4140 - 7s - loss: 0.5659 - acc: 0.7046 - val_loss: 0.5211 - val_acc: 0.7826
Epoch 16/30
4140/4140 - 7s - loss: 0.5504 - acc: 0.7143 - val_loss: 0.5112 - val_acc: 0.7913
Epoch 17/30
4140/4140 - 8s - loss: 0.5405 - acc: 0.7258 - val_loss: 0.5027 - val_acc: 0.7957
Epoch 18/30
4140/4140 - 7s - loss: 0.5340 - acc: 0.7370 - val_loss: 0.4989 - val_acc: 0.8043
Epoch 19/30
4140/4140 - 7s - loss: 0.5344 - acc: 0.7464 - val_loss: 0.4880 - val_acc: 0.8109
Epoch 20/30
4140/4140 - 7s - loss: 0.6019 - acc: 0.7486 - val_loss: 0.4947 - val_acc: 0.8174
Epoch 21/30
4140/4140 - 7s - loss: 0.5299 - acc: 0.7536 - val_loss: 0.

In [64]:
process_results(gl__wobn_hist, gl_wobn_evas)

mean  acc         0.727967
loss        0.518230
val_acc     0.765014
val_loss    0.524105
dtype: float64
std  acc         0.075186
loss        0.097495
val_acc     0.082013
val_loss    0.095106
dtype: float64


media  0.8450000047683716
std  0.022730307373557055


## Pruebas con topologías

### 2 lstm

In [570]:

gc.collect()

20

In [59]:
def double_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', return_sequences=True, input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.LSTM(10, activation='relu', name='2lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
double_hist, double_evas = kfold_train(double_lstm_model, 'double_lstm', batch_size=128, epochs=30, shuffle=False, verbose=2)

W0623 10:33:18.378517 140412910548800 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0623 10:33:18.589042 140412910548800 deprecation.py:323] From /home/suampa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


logs/fit/double_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 4s - loss: 0.6875 - acc: 0.5920 - val_loss: 0.6651 - val_acc: 0.7043
Epoch 2/30
4140/4140 - 2s - loss: 0.6469 - acc: 0.6679 - val_loss: 0.5975 - val_acc: 0.7196
Epoch 3/30
4140/4140 - 2s - loss: 0.5605 - acc: 0.7229 - val_loss: 0.4615 - val_acc: 0.7587
Epoch 4/30
4140/4140 - 2s - loss: 0.5143 - acc: 0.7630 - val_loss: 0.5105 - val_acc: 0.7826
Epoch 5/30
4140/4140 - 2s - loss: 0.5129 - acc: 0.7853 - val_loss: 0.4425 - val_acc: 0.8087
Epoch 6/30
4140/4140 - 2s - loss: 0.4303 - acc: 0.8024 - val_loss: 0.4137 - val_acc: 0.8130
Epoch 7/30
4140/4140 - 2s - loss: 0.4116 - acc: 0.8184 - val_loss: 0.4092 - val_acc: 0.8196
Epoch 8/30
4140/4140 - 2s - loss: 0.3895 - acc: 0.8278 - val_loss: 0.4064 - val_acc: 0.8152
Epoch 9/30
4140/4140 - 2s - loss: 0.3844 - acc: 0.8278 - val_loss: 0.4116 - val_acc: 0.8087
Epoch 10/30
4140/4140 - 4s - loss: 0.3872 - acc: 0.8319 - val_loss: 0.4111 - val_acc: 0.8087
Epoc

W0623 10:37:09.167119 140412910548800 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


logs/fit/double_lstm/kfold2
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 3s - loss: 0.6978 - acc: 0.5816 - val_loss: 0.6812 - val_acc: 0.5652
Epoch 2/30
4140/4140 - 2s - loss: 0.6811 - acc: 0.5838 - val_loss: 0.6809 - val_acc: 0.5609
Epoch 3/30
4140/4140 - 4s - loss: 0.6843 - acc: 0.5783 - val_loss: 0.6806 - val_acc: 0.5609
Epoch 4/30
4140/4140 - 7s - loss: 0.6759 - acc: 0.5807 - val_loss: 0.6762 - val_acc: 0.5609
Epoch 5/30
4140/4140 - 13s - loss: 0.6529 - acc: 0.6092 - val_loss: 0.5834 - val_acc: 0.7022
Epoch 6/30
4140/4140 - 15s - loss: 0.6111 - acc: 0.7466 - val_loss: 0.6187 - val_acc: 0.6652
Epoch 7/30
4140/4140 - 18s - loss: 0.5899 - acc: 0.6954 - val_loss: 0.5374 - val_acc: 0.7174
Epoch 8/30
4140/4140 - 17s - loss: 0.5041 - acc: 0.7490 - val_loss: 0.5173 - val_acc: 0.7261
Epoch 9/30
4140/4140 - 15s - loss: 0.4673 - acc: 0.7696 - val_loss: 0.4995 - val_acc: 0.7261
Epoch 10/30
4140/4140 - 15s - loss: 0.4582 - acc: 0.7720 - val_loss: 0.4889 - val_acc: 0.758

W0623 10:43:25.719881 140412910548800 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


...

logs/fit/double_lstm/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 26s - loss: 0.6979 - acc: 0.6046 - val_loss: 0.6947 - val_acc: 0.6130
Epoch 2/30
4140/4140 - 16s - loss: 0.6841 - acc: 0.6010 - val_loss: 0.6849 - val_acc: 0.5348
Epoch 3/30
4140/4140 - 16s - loss: 0.6632 - acc: 0.6077 - val_loss: 0.6328 - val_acc: 0.5891
Epoch 4/30
4140/4140 - 16s - loss: 0.5912 - acc: 0.6908 - val_loss: 0.5716 - val_acc: 0.6804
Epoch 5/30
4140/4140 - 16s - loss: 0.5136 - acc: 0.7425 - val_loss: 0.5103 - val_acc: 0.7478
Epoch 6/30
4140/4140 - 16s - loss: 0.4747 - acc: 0.7643 - val_loss: 0.4772 - val_acc: 0.7543
Epoch 7/30
4140/4140 - 16s - loss: 0.4304 - acc: 0.7947 - val_loss: 0.4133 - val_acc: 0.8174
Epoch 8/30
4140/4140 - 15s - loss: 0.4274 - acc: 0.8041 - val_loss: 0.4366 - val_acc: 0.7935
Epoch 9/30
4140/4140 - 14s - loss: 0.3952 - acc: 0.8234 - val_loss: 0.3865 - val_acc: 0.8261
Epoch 10/30
4140/4140 - 3s - loss: 0.3778 - acc: 0.8278 - val_loss: 0.3881 - val_acc: 0.

W0623 10:55:11.599130 140412910548800 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


logs/fit/double_lstm/kfold5
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 23s - loss: 0.6895 - acc: 0.5771 - val_loss: 0.6683 - val_acc: 0.5804
Epoch 2/30
4140/4140 - 14s - loss: 0.6541 - acc: 0.5932 - val_loss: 0.6082 - val_acc: 0.6913
Epoch 3/30
4140/4140 - 14s - loss: 0.5702 - acc: 0.7181 - val_loss: 0.5334 - val_acc: 0.7196
Epoch 4/30
4140/4140 - 15s - loss: 0.4872 - acc: 0.7652 - val_loss: 0.4915 - val_acc: 0.7543
Epoch 5/30
4140/4140 - 15s - loss: 0.4436 - acc: 0.7942 - val_loss: 0.4729 - val_acc: 0.7696
Epoch 6/30
4140/4140 - 15s - loss: 0.4053 - acc: 0.8164 - val_loss: 0.4992 - val_acc: 0.7761
Epoch 7/30
4140/4140 - 15s - loss: 0.3776 - acc: 0.8309 - val_loss: 0.5425 - val_acc: 0.7761
Epoch 8/30
4140/4140 - 14s - loss: 0.3613 - acc: 0.8382 - val_loss: 0.4926 - val_acc: 0.7826
Epoch 9/30
4140/4140 - 15s - loss: 0.3482 - acc: 0.8505 - val_loss: 0.4970 - val_acc: 0.7826
Epoch 10/30
4140/4140 - 14s - loss: 0.3384 - acc: 0.8543 - val_loss: 0.5132 - val_acc: 0

Epoch 25/30
4140/4140 - 14s - loss: 0.2717 - acc: 0.8795 - val_loss: 0.5703 - val_acc: 0.8283
Epoch 26/30
4140/4140 - 13s - loss: 0.2655 - acc: 0.8867 - val_loss: 0.5688 - val_acc: 0.8239
Epoch 27/30
4140/4140 - 13s - loss: 0.2898 - acc: 0.8768 - val_loss: 0.6594 - val_acc: 0.8196
Epoch 28/30
4140/4140 - 13s - loss: 0.2845 - acc: 0.8792 - val_loss: 0.6584 - val_acc: 0.8174
Epoch 29/30
4140/4140 - 13s - loss: 0.2616 - acc: 0.8908 - val_loss: 0.6473 - val_acc: 0.8152
Epoch 30/30
4140/4140 - 13s - loss: 0.2670 - acc: 0.8872 - val_loss: 0.7104 - val_acc: 0.8065
100/100 [==============================] - 1s 7ms/sample - loss: 0.5510 - acc: 0.8500
logs/fit/double_lstm/kfold8
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 23s - loss: 0.6902 - acc: 0.5935 - val_loss: 0.6837 - val_acc: 0.6370
Epoch 2/30
4140/4140 - 14s - loss: 0.6800 - acc: 0.6635 - val_loss: 0.6615 - val_acc: 0.6587
Epoch 3/30
4140/4140 - 13s - loss: 0.6424 - acc: 0.6899 - val_loss: 0.5937 - val_acc: 0.6

Epoch 18/30
4140/4140 - 9s - loss: 0.3416 - acc: 0.8490 - val_loss: 0.3414 - val_acc: 0.8565
Epoch 19/30
4140/4140 - 14s - loss: 0.3433 - acc: 0.8505 - val_loss: 0.3405 - val_acc: 0.8543
Epoch 20/30
4140/4140 - 14s - loss: 0.3256 - acc: 0.8606 - val_loss: 0.3410 - val_acc: 0.8543
Epoch 21/30
4140/4140 - 13s - loss: 0.3219 - acc: 0.8630 - val_loss: 0.3371 - val_acc: 0.8543
Epoch 22/30
4140/4140 - 14s - loss: 0.3201 - acc: 0.8708 - val_loss: 0.3397 - val_acc: 0.8543
Epoch 23/30
4140/4140 - 13s - loss: 0.3154 - acc: 0.8681 - val_loss: 0.3373 - val_acc: 0.8565
Epoch 24/30
4140/4140 - 14s - loss: 0.3064 - acc: 0.8749 - val_loss: 0.3401 - val_acc: 0.8522
Epoch 25/30
4140/4140 - 14s - loss: 0.3008 - acc: 0.8785 - val_loss: 0.3473 - val_acc: 0.8543
Epoch 26/30
4140/4140 - 14s - loss: 0.2914 - acc: 0.8829 - val_loss: 0.3422 - val_acc: 0.8522
Epoch 27/30
4140/4140 - 13s - loss: 0.2931 - acc: 0.8838 - val_loss: 0.3450 - val_acc: 0.8522
Epoch 28/30
4140/4140 - 13s - loss: 0.2876 - acc: 0.8841 - va

In [60]:
process_results(double_hist, double_evas)

mean  acc         0.817933
loss        0.382986
val_acc     0.795370
val_loss    0.465998
dtype: float64
std  acc         0.096047
loss        0.128176
val_acc     0.078048
val_loss    0.099289
dtype: float64


media  0.8580000042915344
std  0.013165617325584458


### 3 lstm

In [76]:
def triple_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', return_sequences=True, input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.LSTM(10, activation='relu', name='2lstm', return_sequences=True),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.LSTM(10, activation='relu', name='3lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
triple_hist, triple_evas = kfold_train(triple_lstm_model, 'triple_lstm', batch_size=128, epochs=30, shuffle=False, verbose=2)

logs/fit/triple_lstm/kfold1
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 11s - loss: 0.6891 - acc: 0.5771 - val_loss: 0.6720 - val_acc: 0.5957
Epoch 2/30
4140/4140 - 4s - loss: 0.6544 - acc: 0.5771 - val_loss: 0.5975 - val_acc: 0.5957
Epoch 3/30
4140/4140 - 4s - loss: 0.5850 - acc: 0.5771 - val_loss: 0.5421 - val_acc: 0.5957
Epoch 4/30
4140/4140 - 4s - loss: 0.5308 - acc: 0.6234 - val_loss: 0.5039 - val_acc: 0.7543
Epoch 5/30
4140/4140 - 5s - loss: 0.4928 - acc: 0.7601 - val_loss: 0.4826 - val_acc: 0.8239
Epoch 6/30
4140/4140 - 12s - loss: 0.4694 - acc: 0.8022 - val_loss: 0.5408 - val_acc: 0.7913
Epoch 7/30
4140/4140 - 25s - loss: 0.4790 - acc: 0.7838 - val_loss: 0.4740 - val_acc: 0.8196
Epoch 8/30
4140/4140 - 26s - loss: 0.4321 - acc: 0.8034 - val_loss: 0.4493 - val_acc: 0.8304
Epoch 9/30
4140/4140 - 28s - loss: 0.4016 - acc: 0.8244 - val_loss: 0.4612 - val_acc: 0.8130
Epoch 10/30
4140/4140 - 28s - loss: 0.3995 - acc: 0.8285 - val_loss: 0.4296 - val_acc: 0.800

Epoch 25/30
4140/4140 - 3s - loss: 0.2648 - acc: 0.8928 - val_loss: 0.3556 - val_acc: 0.8609
Epoch 26/30
4140/4140 - 3s - loss: 0.2545 - acc: 0.8976 - val_loss: 0.3571 - val_acc: 0.8696
Epoch 27/30
4140/4140 - 4s - loss: 0.2466 - acc: 0.9022 - val_loss: 0.3539 - val_acc: 0.8587
Epoch 28/30
4140/4140 - 9s - loss: 0.2348 - acc: 0.9104 - val_loss: 0.3601 - val_acc: 0.8587
Epoch 29/30
4140/4140 - 17s - loss: 0.2462 - acc: 0.8995 - val_loss: 0.3502 - val_acc: 0.8652
Epoch 30/30
4140/4140 - 20s - loss: 0.2269 - acc: 0.9140 - val_loss: 0.4028 - val_acc: 0.8609
100/100 [==============================] - 1s 11ms/sample - loss: 0.3085 - acc: 0.9100
logs/fit/triple_lstm/kfold4
Train on 4140 samples, validate on 460 samples
Epoch 1/30
4140/4140 - 59s - loss: 0.6889 - acc: 0.6239 - val_loss: 0.6670 - val_acc: 0.6848
Epoch 2/30
4140/4140 - 20s - loss: 0.6583 - acc: 0.6932 - val_loss: 0.6642 - val_acc: 0.6587
Epoch 3/30
4140/4140 - 6s - loss: 0.6071 - acc: 0.7546 - val_loss: 0.6555 - val_acc: 0.7370


Epoch 18/30
4140/4140 - 17s - loss: 0.2701 - acc: 0.8865 - val_loss: 0.6099 - val_acc: 0.8261
Epoch 19/30
4140/4140 - 3s - loss: 0.2661 - acc: 0.8870 - val_loss: 0.6695 - val_acc: 0.8304
Epoch 20/30
4140/4140 - 3s - loss: 0.2591 - acc: 0.8913 - val_loss: 0.6615 - val_acc: 0.8239
Epoch 21/30
4140/4140 - 3s - loss: 0.2513 - acc: 0.8947 - val_loss: 0.7281 - val_acc: 0.8348
Epoch 22/30
4140/4140 - 3s - loss: 0.2427 - acc: 0.8961 - val_loss: 0.8586 - val_acc: 0.8435
Epoch 23/30
4140/4140 - 3s - loss: 0.2342 - acc: 0.9019 - val_loss: 1.0567 - val_acc: 0.8283
Epoch 24/30
4140/4140 - 3s - loss: 0.2258 - acc: 0.9094 - val_loss: 1.2122 - val_acc: 0.8283
Epoch 25/30
4140/4140 - 3s - loss: 0.2198 - acc: 0.9114 - val_loss: 0.9960 - val_acc: 0.8326
Epoch 26/30
4140/4140 - 5s - loss: 0.2109 - acc: 0.9171 - val_loss: 1.0688 - val_acc: 0.8239
Epoch 27/30
4140/4140 - 11s - loss: 0.2045 - acc: 0.9174 - val_loss: 1.0619 - val_acc: 0.8217
Epoch 28/30
4140/4140 - 19s - loss: 0.2018 - acc: 0.9208 - val_loss:

Epoch 11/30
4140/4140 - 20s - loss: 0.5338 - acc: 0.7244 - val_loss: 0.5951 - val_acc: 0.7304
Epoch 12/30
4140/4140 - 7s - loss: 0.5177 - acc: 0.7548 - val_loss: 0.5565 - val_acc: 0.7630
Epoch 13/30
4140/4140 - 3s - loss: 0.4891 - acc: 0.7932 - val_loss: 0.5382 - val_acc: 0.7609
Epoch 14/30
4140/4140 - 3s - loss: 0.4682 - acc: 0.8068 - val_loss: 0.4976 - val_acc: 0.7826
Epoch 15/30
4140/4140 - 3s - loss: 0.4546 - acc: 0.8162 - val_loss: 0.4884 - val_acc: 0.7870
Epoch 16/30
4140/4140 - 3s - loss: 0.4464 - acc: 0.8283 - val_loss: 0.4792 - val_acc: 0.7957
Epoch 17/30
4140/4140 - 3s - loss: 0.4319 - acc: 0.8370 - val_loss: 0.4670 - val_acc: 0.7957
Epoch 18/30
4140/4140 - 3s - loss: 0.4224 - acc: 0.8348 - val_loss: 0.4967 - val_acc: 0.7848
Epoch 19/30
4140/4140 - 4s - loss: 0.4101 - acc: 0.8372 - val_loss: 0.4682 - val_acc: 0.7804
Epoch 20/30
4140/4140 - 9s - loss: 0.3796 - acc: 0.8428 - val_loss: 0.4527 - val_acc: 0.7696
Epoch 21/30
4140/4140 - 16s - loss: 0.3562 - acc: 0.8425 - val_loss: 

In [73]:
process_results(tiple_hist, triple_evas)

mean  acc         0.823099
loss        0.378627
val_acc     0.797746
val_loss    0.455723
dtype: float64
std  acc         0.092829
loss        0.128274
val_acc     0.075640
val_loss    0.094572
dtype: float64


media  0.8479999959468841
std  0.022010101773470788


In [ ]:
model_lstm = Sequential()
model_2lstm = Sequential()
model_2dcnn = Sequential()
model_2dcnnLstm = Sequential()
model_cnn = Sequential()
model_bidi = Sequential()

model_lstm.add(LSTM(28, activation='relu', input_shape=(elements, features), name='lstm'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(polarity_dim, activation='sigmoid', name='dense'))
model_lstm.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

model_2lstm.add(LSTM(neurons, activation='tanh', input_shape=input_shape, return_sequences=True, name='lstm'))
model_2lstm.add(Dropout(0.5))
model_2lstm.add(LSTM(neurons, activation='tanh'))
model_2lstm.add(Dropout(0.5))
model_2lstm.add(Dense(polarity_dim, activation='softmax', name='dense'))
model_2lstm.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

model_2dcnn.add(Reshape((1, timesteps, features), input_shape=input_shape))
model_2dcnn.add(Conv2D(128, (4, 300), padding='same', name='conv_layer'))
model_2dcnn.add(Activation('relu'))
model_2dcnn.add(MaxPooling2D(pool_size=(2,2), strides=None))
model_2dcnn.add(Flatten())
model_2dcnn.add(Dropout(0.5))
model_2dcnn.add(Dense(polarity_dim, activation='softmax', name='dense'))
model_2dcnn.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

model_2dcnnLstm.add(Reshape((1, timesteps, features), input_shape=input_shape))
model_2dcnnLstm.add(Conv2D(128, (4, 300), padding='same', name='conv_layer'))
model_2dcnnLstm.add(Activation('relu'))
model_2dcnnLstm.add(MaxPooling2D(pool_size=(2,2), strides=None))
model_2dcnnLstm.add(TimeDistributed(Flatten()))
model_2dcnnLstm.add(LSTM(neurons))
model_2dcnnLstm.add(Dropout(0.5))
model_2dcnnLstm.add(Dense(polarity_dim, activation='softmax', name='dense'))
model_2dcnnLstm.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

model_cnn.add(Convolution1D(128, 3, padding='same', name='conv_layer', input_shape=input_shape))
model_cnn.add(Activation('tanh'))
model_cnn.add(MaxPooling1D(4))
model_cnn.add(LSTM(neurons, dropout=0.5))
model_cnn.add(Dense(polarity_dim, activation='softmax', name='dense'))
model_cnn.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

model_bidi.add(Bidirectional(LSTM(neurons), input_shape=input_shape))
model_bidi.add(Dense(polarity_dim, activation='softmax', name='dense'))
model_bidi.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])


## Train

In [ ]:
monitor = 'val_loss'
patience = 5
cbks = [callbacks.EarlyStopping(monitor=monitor, patience=patience)]

#### Regularización de pesos

In [ ]:
pipeline = {
    'lstm': model_lstm,
    '2lstm': model_2lstm,
    '2dcnn': model_2dcnn,
    '2dcnn+lstm': model_2dcnnLstm,
    'cnn+lstm': model_cnn,
    'bidirectionalLstm': model_bidi
}

In [ ]:
def create_model_lstm_reg3():
    model = Sequential()
    model.add(LSTM(10, return_sequences=True, recurrent_initializer='normal', kernel_initializer='normal', activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), name='lstm'))
    model.add(Dropout(0.2))
    model.add(LSTM(10))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense'))
    optimizer = Adam(lr=0.001, decay=0.01)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model
model = create_model_lstm_reg3()
hist_reg3 = model.fit(x_train, y_train, shuffle=False, validation_split=0.2, verbose=2, batch_size=128, epochs=30)
# reg3_hist, reg3_evas = kfold_train(create_reg_model3, batch_size=128, epochs=30, shuffle=False, verbose=2)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
pd.DataFrame(hist_reg3.history)[['loss', 'val_loss']].iplot()

In [ ]:
results = {}
with tqdm(total=len(clasificadores) * 10) as pbar:
    for c in clasificadores:
        results[c] = { 'real': {}, 'predicted': {} }
        i = 0
        for train_index, test_index in kf.split(train_corpus.content):
            train_x = X[train_index]
            train_y = Y[train_index]
            test_x = X[test_index]
            test_y = Y[test_index]
            print(train_y)
            if c == 'bidiLstm':
                train_x = [train_x, np.flipud(train_x)]
                
            pipeline[c].fit(train_x, train_y, batch_size=64, callbacks=cbks, epochs=1000, validation_split=0.25, shuffle=False, verbose=1)

            predicted = pipeline[c].predict(test_x)

            results[c]['real'][i] = test_y.tolist()
            results[c]['predicted'][i] = predicted.tolist()
            i = i + 1

            pbar.update(1)

    

In [ ]:
results

In [ ]:
pd.DataFrame(results).to_pickle('data/results/'+name+'/cine/' + base_dir + '/' + name + '.pkl')